In [5]:
import pandas

In [6]:
from pyspark.sql import SparkSession 
import pyspark.sql.functions as F

In [7]:
spark = SparkSession.builder.master("local[*]").appName("pyspark_migration").getOrCreate()

In [8]:
matrix = [

[' Ricky', 4879320143, '2022-09-05 11:23:59', True],

['Simon', 4879320143, '2022-09-05 11:23:59', True],

['Sergio', 4879320143, '2022-09-05 11:23:59', True],

['Pedrito', 4879320143, '2022-09-07 11:23:59', True],

['jJuenito', 4879320143, '2022-09-08 11:23:59', True],

['Pepe', 4879320143, '2022-09-09 11:23:59', True],

['Liusa', 4879320143, '2022-09-05 11:23:59', True],

]

columns = ['name', 'dni', 'date', 'is_human']

In [9]:
df = spark.createDataFrame(matrix, columns)

In [10]:
df_with_constant = df.withColumn( "constant\_value", F.lit(1))

df_with_constant.show()

+--------+----------+-------------------+--------+---------------+
|    name|       dni|               date|is_human|constant\_value|
+--------+----------+-------------------+--------+---------------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|              1|
|   Simon|4879320143|2022-09-05 11:23:59|    true|              1|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|              1|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|              1|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|              1|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|              1|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|              1|
+--------+----------+-------------------+--------+---------------+



In [11]:
# 1. `lit`: Ajouter une colonne avec une valeur constante

In [12]:
df_lit = df.select(F.lit(2).alias('test'))
df_lit.show()


+----+
|test|
+----+
|   2|
|   2|
|   2|
|   2|
|   2|
|   2|
|   2|
+----+



In [13]:
# 2. `col` et `when`: Créer une colonne avec une condition

In [14]:


df_col = df.withColumn(
    "new col",
    F.when(F.expr("name LIKE '%to'"), "found")
     .when(df["name"] == "Alice", "Alice found")
     .otherwise("not found")
)

df_col.show()


+--------+----------+-------------------+--------+---------+
|    name|       dni|               date|is_human|  new col|
+--------+----------+-------------------+--------+---------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|not found|
|   Simon|4879320143|2022-09-05 11:23:59|    true|not found|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|not found|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|    found|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|    found|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|not found|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|not found|
+--------+----------+-------------------+--------+---------+



In [15]:
# 3. `concat`: Combiner des colonnes en une seule chaîne

In [16]:
df_concat = df.select(F.concat(df.is_human, df.dni).alias("humain"), "name", "dni", "date")
df_concat.show()

+--------------+--------+----------+-------------------+
|        humain|    name|       dni|               date|
+--------------+--------+----------+-------------------+
|true4879320143|   Ricky|4879320143|2022-09-05 11:23:59|
|true4879320143|   Simon|4879320143|2022-09-05 11:23:59|
|true4879320143|  Sergio|4879320143|2022-09-05 11:23:59|
|true4879320143| Pedrito|4879320143|2022-09-07 11:23:59|
|true4879320143|jJuenito|4879320143|2022-09-08 11:23:59|
|true4879320143|    Pepe|4879320143|2022-09-09 11:23:59|
|true4879320143|   Liusa|4879320143|2022-09-05 11:23:59|
+--------------+--------+----------+-------------------+



In [17]:
# 4. `month`, `year`, `quarter`: Extraire le mois, l'année et le trimestre
#df = df.withColumn('date', F.to_date(df['date'], 'yyyy-MM-dd'))

In [18]:
df_quarter = df.withColumn('quarter', F.quarter(df['date']))
df_quarter.show()

+--------+----------+-------------------+--------+-------+
|    name|       dni|               date|is_human|quarter|
+--------+----------+-------------------+--------+-------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|      3|
|   Simon|4879320143|2022-09-05 11:23:59|    true|      3|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|      3|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|      3|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|      3|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|      3|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|      3|
+--------+----------+-------------------+--------+-------+



In [19]:
df_year = df.withColumn('year', F.year(df['date']))
df_year.show()

+--------+----------+-------------------+--------+----+
|    name|       dni|               date|is_human|year|
+--------+----------+-------------------+--------+----+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|2022|
|   Simon|4879320143|2022-09-05 11:23:59|    true|2022|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|2022|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|2022|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|2022|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|2022|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|2022|
+--------+----------+-------------------+--------+----+



In [20]:
df_month = df.withColumn("month", F.date_format('date', 'MMMM'))
    #.withColumn("year", F.date_format('date', 'y')) \
   # .withColumn("quarter", F.date_format('date', 'QQQQ'))

df_month.show()

+--------+----------+-------------------+--------+---------+
|    name|       dni|               date|is_human|    month|
+--------+----------+-------------------+--------+---------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|September|
|   Simon|4879320143|2022-09-05 11:23:59|    true|September|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|September|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|September|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|September|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|September|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|September|
+--------+----------+-------------------+--------+---------+



In [21]:
# 5. `abs` et `floor`: Appliquer des fonctions mathématiques

In [22]:

# A REFAIRE MARCHE PAS

data_abs = [(-8,), (5,), (-4,)]
col = ["val"]

df_abs = spark.createDataFrame(data_abs, col)

df_result = df_abs.withColumn("absolute_value", F.floor(df_abs["val"]))

df_result.show()

+---+--------------+
|val|absolute_value|
+---+--------------+
| -8|            -8|
|  5|             5|
| -4|            -4|
+---+--------------+



In [23]:
data_floor = [(-8,), (5,), (-4,)]
col = ["val"]

# Création du DataFrame
df_floor = spark.createDataFrame(data_abs, col)

# Application de la fonction abs() sur la colonne 'val'
df_result = df_floor.withColumn("floor_value", F.floor(df_floor["val"]))

# Afficher le résultat
df_result.show()

+---+-----------+
|val|floor_value|
+---+-----------+
| -8|         -8|
|  5|          5|
| -4|         -4|
+---+-----------+



In [24]:
# 6. `concat_ws`: Concaténer avec un séparateur

In [25]:
df_concat_ws = df.select(F.concat_ws('_', df.name, df.dni).alias('n'))

df_concat_ws.show()

+-------------------+
|                  n|
+-------------------+
|   Ricky_4879320143|
|   Simon_4879320143|
|  Sergio_4879320143|
| Pedrito_4879320143|
|jJuenito_4879320143|
|    Pepe_4879320143|
|   Liusa_4879320143|
+-------------------+



In [26]:
# 7. `array_distinct`, `collect_list`, `collect_set`: Utiliser des fonctions d'agrégation sur des arrays

In [27]:
df_array_distinct = spark.createDataFrame([([1, 2, 3, 2],), ([4, 5, 5, 4],)], ['data'])

df_array_distinct.show()

p = df_array_distinct.select(F.array_distinct(df_array_distinct.data))

p.show()

+------------+
|        data|
+------------+
|[1, 2, 3, 2]|
|[4, 5, 5, 4]|
+------------+

+--------------------+
|array_distinct(data)|
+--------------------+
|           [1, 2, 3]|
|              [4, 5]|
+--------------------+



In [28]:
data_collect = [('James','Java'),
  ('James','Python'),
  ('James','Python'),
  ('Anna','PHP'),
  ('Anna','Javascript'),
  ('Maria','Java'),
  ('Maria','C++'),
  ('James','Scala'),
  ('Anna','PHP'),
  ('Anna','HTML')
]

df_collect = spark.createDataFrame(data_collect,schema=["name","languages"])

df_collect.show()

#******************************************************************
#-----------------------collect_list()----------------------------
#collect_list permet de colecter les multiples données d'une personne, pour eviter d'avoir plusieurs colonnes 'James'
#******************************************************************



df_colist = df_collect.groupBy("name").agg(F.collect_list("languages") \
                                           .alias("languages"))

df_colist.show(truncate=False)

#******************************************************************
#-----------------------collect_set()----------------------------
#ça marche comme collect_list sauf que set elimine les resultats dupliquer en une unique valeur
#******************************************************************

df_colset = df_collect.groupBy("name").agg(F.collect_set("languages") \
                                           .alias("languages"))

df_colset.show()

+-----+----------+
| name| languages|
+-----+----------+
|James|      Java|
|James|    Python|
|James|    Python|
| Anna|       PHP|
| Anna|Javascript|
|Maria|      Java|
|Maria|       C++|
|James|     Scala|
| Anna|       PHP|
| Anna|      HTML|
+-----+----------+

+-----+-----------------------------+
|name |languages                    |
+-----+-----------------------------+
|James|[Java, Python, Python, Scala]|
|Anna |[PHP, Javascript, PHP, HTML] |
|Maria|[Java, C++]                  |
+-----+-----------------------------+

+-----+--------------------+
| name|           languages|
+-----+--------------------+
|James|[Scala, Java, Pyt...|
| Anna|[PHP, Javascript,...|
|Maria|         [Java, C++]|
+-----+--------------------+



In [29]:
# 8. `split` et `trim`: Nettoyer et diviser des chaînes


In [41]:
#******************************************************************
#-----------------------SPLIT()---------------------------
#******************************************************************

data_split = [('James','','Smith','1991-04-01'),
  ('Michael','Rose','','2000-05-19'),
  ('Robert','','Williams','1978-09-05'),
  ('Maria','Anne','Jones','1967-12-01'),
  ('Jen','Mary','Brown','1980-02-17')
]

columns=["firstname","middlename","lastname","dob"]
df_split=spark.createDataFrame(data_split,columns)
#df_split.printSchema()
df_split.show(truncate=False)

#creer une fonction pour les reutiliser
split_col = F.split(df_split['dob'], '-')
df_split_split = df_split.withColumn('year', split_col.getItem(0)) \
       .withColumn('month', split_col.getItem(1)) \
       .withColumn('day', split_col.getItem(2))
df_split_split.show(truncate=False) 


+---------+----------+--------+----------+
|firstname|middlename|lastname|dob       |
+---------+----------+--------+----------+
|James    |          |Smith   |1991-04-01|
|Michael  |Rose      |        |2000-05-19|
|Robert   |          |Williams|1978-09-05|
|Maria    |Anne      |Jones   |1967-12-01|
|Jen      |Mary      |Brown   |1980-02-17|
+---------+----------+--------+----------+

+---------+----------+--------+----------+----+-----+---+
|firstname|middlename|lastname|dob       |year|month|day|
+---------+----------+--------+----------+----+-----+---+
|James    |          |Smith   |1991-04-01|1991|04   |01 |
|Michael  |Rose      |        |2000-05-19|2000|05   |19 |
|Robert   |          |Williams|1978-09-05|1978|09   |05 |
|Maria    |Anne      |Jones   |1967-12-01|1967|12   |01 |
|Jen      |Mary      |Brown   |1980-02-17|1980|02   |17 |
+---------+----------+--------+----------+----+-----+---+



In [32]:
#******************************************************************
#-----------------------TRIM()---------------------------
#Enlevé les espace blanc en debut ou fin de caractère de col, je vais utiliser "dob"
#meme si j'ai l'impression qu'il n'en a pas besoin a cause de "jJuanito"
#******************************************************************
df.show()
df_trim = df.withColumn("name", F.trim(df.name))
df_trim.show()

+--------+----------+-------------------+--------+
|    name|       dni|               date|is_human|
+--------+----------+-------------------+--------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|
|   Simon|4879320143|2022-09-05 11:23:59|    true|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|
|    Pepe|4879320143|2022-09-09 11:23:59|    true|
|   Liusa|4879320143|2022-09-05 11:23:59|    true|
+--------+----------+-------------------+--------+

+--------+----------+-------------------+--------+
|    name|       dni|               date|is_human|
+--------+----------+-------------------+--------+
|   Ricky|4879320143|2022-09-05 11:23:59|    true|
|   Simon|4879320143|2022-09-05 11:23:59|    true|
|  Sergio|4879320143|2022-09-05 11:23:59|    true|
| Pedrito|4879320143|2022-09-07 11:23:59|    true|
|jJuenito|4879320143|2022-09-08 11:23:59|    true|
|    Pepe|4879320143|2022-09-0

In [ ]:
# 9. `explode`: Diviser une colonne de tableau en lignes distinctes

In [35]:
#******************************************************************
#-----------------------Explode()---------------------------
#Fonction qui me fait penser à "collect_list()", parce que explode va remettre les col key - value qui sont presente plusieurs fois
#je recup "df_colist", qui à rassembler toute les valeur en communs d'une clé pour les mettre dans une meme row "value"
#******************************************************************

#je regarde l'etat de ce df pour être sûr
df_colist.show()

df_explode = df_colist.select(df_colist.name, F.explode(df_colist.languages))

df_explode.show()
#ptn j'suis une star ! 

+-----+--------------------+
| name|           languages|
+-----+--------------------+
|James|[Java, Python, Py...|
| Anna|[PHP, Javascript,...|
|Maria|         [Java, C++]|
+-----+--------------------+

+-----+----------+
| name|       col|
+-----+----------+
|James|      Java|
|James|    Python|
|James|    Python|
|James|     Scala|
| Anna|       PHP|
| Anna|Javascript|
| Anna|       PHP|
| Anna|      HTML|
|Maria|      Java|
|Maria|       C++|
+-----+----------+



In [ ]:
# 10. `lower`: Convertir des chaînes en minuscules

In [38]:
df_upper = df_explode.select(df_explode.name, F.upper(df_explode.name))
df_upper.show()

df_lower = df_explode.select(df_upper.name, F.lower(df_upper.name))

df_lower.show()

#celui la je suis fière de moi je l'ai fait sans regarde LA DOC , "just sayin'!"



+-----+-----------+
| name|upper(name)|
+-----+-----------+
|James|      JAMES|
|James|      JAMES|
|James|      JAMES|
|James|      JAMES|
| Anna|       ANNA|
| Anna|       ANNA|
| Anna|       ANNA|
| Anna|       ANNA|
|Maria|      MARIA|
|Maria|      MARIA|
+-----+-----------+

+-----+-----------+
| name|lower(name)|
+-----+-----------+
|James|      james|
|James|      james|
|James|      james|
|James|      james|
| Anna|       anna|
| Anna|       anna|
| Anna|       anna|
| Anna|       anna|
|Maria|      maria|
|Maria|      maria|
+-----+-----------+



In [ ]:
# 11. `select`: Sélectionner des colonnes spécifiques

In [43]:
#Je vais utiliser le split et select le day month year

df_split_split.show()

df_split_split.select('year', 'month', 'day').show() #OK NICEEEEUUUUU




+---------+----------+--------+----------+----+-----+---+
|firstname|middlename|lastname|       dob|year|month|day|
+---------+----------+--------+----------+----+-----+---+
|    James|          |   Smith|1991-04-01|1991|   04| 01|
|  Michael|      Rose|        |2000-05-19|2000|   05| 19|
|   Robert|          |Williams|1978-09-05|1978|   09| 05|
|    Maria|      Anne|   Jones|1967-12-01|1967|   12| 01|
|      Jen|      Mary|   Brown|1980-02-17|1980|   02| 17|
+---------+----------+--------+----------+----+-----+---+

+----+-----+---+
|year|month|day|
+----+-----+---+
|1991|   04| 01|
|2000|   05| 19|
|1978|   09| 05|
|1967|   12| 01|
|1980|   02| 17|
+----+-----+---+



In [45]:
## 12. `withColumn`: Créer une nouvelle colonne

df_newcol = df_split_split.withColumn("is_dinosor", F.lit('ofc you are dinosor')).show()

#ca s'écrit pas comme ca dinosor en anglais 

+---------+----------+--------+----------+----+-----+---+-------------------+
|firstname|middlename|lastname|       dob|year|month|day|         is_dinosor|
+---------+----------+--------+----------+----+-----+---+-------------------+
|    James|          |   Smith|1991-04-01|1991|   04| 01|ofc you are dinosor|
|  Michael|      Rose|        |2000-05-19|2000|   05| 19|ofc you are dinosor|
|   Robert|          |Williams|1978-09-05|1978|   09| 05|ofc you are dinosor|
|    Maria|      Anne|   Jones|1967-12-01|1967|   12| 01|ofc you are dinosor|
|      Jen|      Mary|   Brown|1980-02-17|1980|   02| 17|ofc you are dinosor|
+---------+----------+--------+----------+----+-----+---+-------------------+



In [54]:
# 13. `withColumnRenamed`: Renommer une colonne

#df_renamed = df_newcol.withColumnRenamed("is_dinosor", "dinosaurio")

df_newcol.printSchema()


AttributeError: 'NoneType' object has no attribute 'printSchema'